## Primera implementación del método de involuciones 

In [1]:
using PyCall
using LaTeXStrings
@pyimport numpy as np
using SymPy
using Roots
using Plots

┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:410
└ @ PyCall /home/eve/.julia/packages/PyCall/BcTLp/src/PyCall.jl:410


Definimos una función para componer n veces (tendremos 3 opciones)

In [2]:
composition(f, n) = ∘(ntuple(_ -> f, n)...) # donde f es una funcion de una variable y composition regresa una funcion

composition (generic function with 1 method)

In [3]:
compose(f, n) = reduce(∘, ntuple(_ -> f, n)) # donde f es una funcion de 1 variable

compose (generic function with 1 method)

In [4]:
function composeNT(f, n)
    function (x)  # <- Esto indica que es una funcion anonima
        val = f(x)
        for _ in 2:n
            val = f(val)
        end
        return val
    end
end


composeNT (generic function with 1 method)

In [5]:
function MapeoX(x,y,a,b)
    vx = x+a*(1-(y-b*sin(2*pi*x))^2)
    return vx
end

function MapeoY(x,y,a,b)
    vy = y-b*sin(2*pi*x)
    return vy
end

MapeoY (generic function with 1 method)

In [6]:
MapeoXA(y) = MapeoX(0.0,y,0.1,0.1)
MapeoYA(y) = MapeoY(0.0,y,0.1,0.1)

MapeoYA (generic function with 1 method)

In [7]:
Mapeo2XA(y) = MapeoXA(MapeoXA(y))
Mapeo2YA(y) = MapeoYA(MapeoYA(y))

Mapeo2YA (generic function with 1 method)

In [8]:
function periodicoSimY(fx, fy, intervalo)
    cerosy1 = find_zeros(fx,intervalo[1],intervalo[2])
    cerosy2 = find_zeros(fy,intervalo[1],intervalo[2])
    return [cerosy1,cerosy2]
end

periodicoSimY (generic function with 1 method)

In [9]:
cerosY = periodicoSimY(Mapeo2XA, Mapeo2YA, [-4.0,4.0])

2-element Array{Array{Float64,1},1}:
 [-3.3166247903554, 3.3166247903554]
 [0.0]

In [10]:
function simetria1(y,a)
    x = mod((a/2)*(1-y^2),1)
    return x
end

simetria1 (generic function with 1 method)

In [11]:
simetria1A(y) = simetria1(y,0.1)

simetria1A (generic function with 1 method)

In [12]:
function periodicoSimX(fx,fy,simetria,cerosy1,cerosy2)
    cerosx1 = [simetria(cerosy1[i]) for i in 1:length(cerosy1)]
    cerosx2 = [simetria(cerosy2[i]) for i in 1:length(cerosy2)]
    return [cerosx1, cerosx2]
end

periodicoSimX (generic function with 1 method)

In [13]:
periodicoSimX(Mapeo2XA,Mapeo2YA,simetria1A,cerosY[1],cerosY[2])

2-element Array{Array{Float64,1},1}:
 [0.5, 0.5]
 [0.05]

In [14]:
mapeo2X(x)=composition(MapeoXA,20)(x) #esto esta mal debe ser funcion de y 
mapeo2Y(y)=composition(MapeoYA,20)(y)

mapeo2Y (generic function with 1 method)

In [15]:
cerosY = periodicoSimY(mapeo2X,mapeo2Y,[-18.0,18.0])

2-element Array{Array{Float64,1},1}:
 [-10.09898409327636, 10.09898409327636]
 [0.0]

In [16]:
periodicoSimX(mapeo2X,mapeo2Y,simetria1A,cerosY[1],cerosY[2])

2-element Array{Array{Float64,1},1}:
 [0.9505260141875524, 0.9505260141875524]
 [0.05]

* Como en general vamos a dar un mapeo y queremos que se sustituyan las simetrias y se vayan encontrando los puntos periodicos queremos juntar lo anterior 

Poe ejemplo imaginemos que tenemos el mapeo siguiente

In [ ]:
function MapeoX(x,y,a,b)
    vx = x+a*(1-(y-b*sin(2*pi*x))^2)
    return vx
end

function MapeoY(x,y,a,b)
    vy = y-b*sin(2*pi*x)
    return vy
end

* Despues nos dan los parametros y la simetria 

a=0.1
b=0.1

sim1 = funcion de y


* Necesitamos hacer un nuevo mapeo interno con esas caracteristicas


In [77]:
function substitutionMap(fx,fy,a,b,sim)
    MapeoXA(y) = MapeoX(sim(y),y,a,b)
    MapeoYA(y) = MapeoY(sim(y),y,a,b)
    return [MapeoXA,MapeoYA]
end

substitutionMap (generic function with 1 method)

In [85]:
simetria1(y)= 0.0
simetria2(y)=(1-y^2)/2

simetria2 (generic function with 1 method)

In [86]:
Maps1 = substitutionMap(MapeoX,MapeoY,0.1,0.1,simetria1)

2-element Array{Function,1}:
 var"#MapeoXA#17"{Float64,Float64,typeof(simetria1)}(0.1, 0.1, simetria1)
 var"#MapeoYA#18"{Float64,Float64,typeof(simetria1)}(0.1, 0.1, simetria1)

In [87]:
Maps1[2]

(::var"#MapeoYA#18"{Float64,Float64,typeof(simetria1)}) (generic function with 1 method)

* Estas funciones son las que iteramos despues 


In [88]:
mapeo2X(y)=composition(Maps1[1],20)(y)
mapeo2Y(y)=composition(Maps1[2],20)(y)

mapeo2Y (generic function with 1 method)

In [91]:
cerosY = periodicoSimY(mapeo2X,mapeo2Y,[-18.0,18.0])

2-element Array{Array{Float64,1},1}:
 [-10.09898409327636, 10.09898409327636]
 [0.0]

In [93]:
periodicoSimX(mapeo2X,mapeo2Y,simetria1,cerosY[1],cerosY[2])

2-element Array{Array{Float64,1},1}:
 [0.0, 0.0]
 [0.0]

* Que pasa si la condicion de simetria es nua funcion?

* Veamos si se puede sustituir

In [99]:
Maps2 = substitutionMap(MapeoX,MapeoY,0.1,0.1,simetria2)

2-element Array{Function,1}:
 var"#MapeoXA#17"{Float64,Float64,typeof(simetria2)}(0.1, 0.1, simetria2)
 var"#MapeoYA#18"{Float64,Float64,typeof(simetria2)}(0.1, 0.1, simetria2)

In [100]:
mapeo3X(y)=composition(Maps2[1],20)(y)
mapeo3Y(y)=composition(Maps2[2],20)(y)

mapeo3Y (generic function with 1 method)

In [101]:
cerosY2 = periodicoSimY(mapeo3X,mapeo3Y,[-1.0,1.0])

2-element Array{Array{Float64,1},1}:
 []
 [2.449293598294706e-16]

In [102]:
periodicoSimX(mapeo3X,mapeo3Y,simetria2,cerosY2[1],cerosY2[2])

2-element Array{Array{Float64,1},1}:
 []
 [0.5]

* Ahora juntamos todo en una funcion o dos

In [105]:
function PeriodicosS(fx,fy,simetria,a,b,n,intervalo)
    Maps = substitutionMap(fx,fy,a,b,simetria)
    mapeoX(y)=composition(Maps[1],n)(y)
    mapeoY(y)=composition(Maps[2],n)(y)
    cerosY = periodicoSimY(mapeoX,mapeoY,[intervalo[1],intervalo[2]])
    cerosX = periodicoSimX(mapeoX,mapeoY,simetria,cerosY[1],cerosY[2])
    return [cerosX,cerosY]
end

PeriodicosS (generic function with 1 method)

In [110]:
PeriodicosS(MapeoX,MapeoY,simetria2,0.1,0.1,20,[-1,1])

2-element Array{Array{Array{Float64,1},1},1}:
 [[], [0.5]]
 [[], [2.449293598294706e-16]]

In [111]:
PeriodicosS(MapeoX,MapeoY,simetria1,0.1,0.1,20,[-19,19])

2-element Array{Array{Array{Float64,1},1},1}:
 [[0.0, 0.0], [0.0]]
 [[-10.09898409327636, 10.09898409327636], [0.0]]

In [116]:
push!(LOAD_PATH,"/home/eve/Documents/Matematicas/Proyectos/periodic_orbits/PeriodicosSymmetries")

6-element Array{String,1}:
 "@"
 "@v#.#"
 "@stdlib"
 "/home/evelyn/Documentos/Matematicas/Proyectos/periodic_orbits/PeriodicosSymmetries"
 "/home/eve/Documents/Matematicas/Proyectos/periodic_orbits/PeriodicosSymmetries"
 "/home/eve/Documents/Matematicas/Proyectos/periodic_orbits/PeriodicosSymmetries"

In [117]:
using PeriodicosSymmetries

ArgumentError: ArgumentError: Package PeriodicosSymmetries not found in current path:
- Run `import Pkg; Pkg.add("PeriodicosSymmetries")` to install the PeriodicosSymmetries package.
